In [ ]:
submit5 xgboost con mismos features que RF sin ht Sklearn LogLoss: 8.09987
0.60068



In [112]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

In [2]:
path_dataset = 'Train_TP2_Datos_2020-2C.csv'
train = pd.read_csv(path_dataset)

In [3]:
path_dataset = 'Test_TP2_Datos_2020-2C.csv'
test= pd.read_csv(path_dataset)

In [4]:
train['Stage']= (train['Stage'] == 'Closed Won') +0

In [5]:
train.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,Q2,2016,NaT,10,EUR,5272800.0,EUR,5272800.0,0,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,Q1,2016,NaT,0,EUR,48230.0,EUR,48230.0,1,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,Q1,2016,NaT,0,USD,83865.6,USD,83865.6,1,Prod_Category_A_None
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,14,USD,7421881.5,USD,7421881.5,0,Prod_Category_A_None
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,25,USD,13357192.5,USD,13357192.5,0,Prod_Category_A_None


In [6]:
# Import KFold
from sklearn.model_selection import KFold

In [7]:
def test_mean_target_en(train, test, target, categorical, alpha=5):
    # Calculate global mean on the train data
    global_mean = train[target].mean()
    
    # Group by the categorical feature and calculate its properties
    train_groups = train.groupby(categorical)
    category_sum = train_groups[target].sum()
    category_size = train_groups.size()
    
    # Calculate smoothed mean target statistics
    train_statistics = (category_sum + global_mean * alpha) / (category_size + alpha)
   
    # Apply statistics to the test data and fill new categories
    test_feature = test[categorical].map(train_statistics).fillna(global_mean)
    return test_feature.values

In [8]:
def train_mean_target_encoding(train, target, categorical, alpha=5):
    # Create 5-fold cross-validation
    kf = KFold(n_splits=5, random_state=123, shuffle=True)
    train_feature = pd.Series(index=train.index)
    
    # For each folds split
    for train_index, test_index in kf.split(train):
        cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
      
        # Calculate out-of-fold statistics and apply to cv_test
        cv_test_feature = test_mean_target_encoding(cv_train, cv_test, target, categorical, alpha)
        
        # Save new feature for this particular fold
        train_feature.iloc[test_index] = cv_test_feature       
    return train_feature.values

In [9]:
def mean_target_encoding(train, test, target, categorical, alpha=5):
  
    # Get the train feature
    train_feature = train_mean_target_encoding(train, target, categorical, alpha)
  
    # Get the test feature
    test_feature = test_mean_target_encoding(train, test, target, categorical, alpha)
    
    # Return new features to add to the model
    return train_feature, test_feature

In [10]:
# Create mean target encoded feature
train['Region_enc'], test['Region_enc'] = mean_target_encoding(train=train,
                                                                     test=test,
                                                                     target='Stage',
                                                                     categorical='Region',
                                                                     alpha=10)

<ipython-input-8-5785e70fe0cb>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  train_feature = pd.Series(index=train.index)


In [11]:
# Look at the encoding
print(test[['Region', 'Region_enc']].drop_duplicates())

         Region  Region_enc
0          EMEA    0.529873
8      Americas    0.454772
17         APAC    0.581181
46        Japan    0.679034
65  Middle East    0.286728


In [12]:
# Create mean target encoded feature
train['Territory_enc'], test['Territory_enc'] = mean_target_encoding(train=train,
                                                                     test=test,
                                                                     target='Stage',
                                                                     categorical='Territory',
                                                                     alpha=10)

<ipython-input-8-5785e70fe0cb>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  train_feature = pd.Series(index=train.index)


In [13]:
# Look at the encoding
print(test[['Territory', 'Territory_enc']].drop_duplicates())

           Territory  Territory_enc
0            Germany       0.607935
8         NE America       0.343669
9         NW America       0.620168
17         Australia       0.576047
46             Japan       0.798300
47          Thailand       0.533471
53         Singapore       0.804047
54             India       0.463523
59            Poland       0.380360
63            France       0.323802
65               KSA       0.505443
68              None       0.606833
85       Netherlands       0.483656
88        Central US       0.415247
89        SW America       0.239005
94        SE America       0.382953
109            Spain       0.671314
115          Denmark       0.513025
142      Philippines       0.400212
152            Malta       0.664074
159   United Kingdom       0.395124
236        Indonesia       0.369666
244          Belgium       0.652967
254         Cambodia       0.544656
271          Andorra       0.562518
289     Saudi Arabia       0.414074
310          Austria       0

In [14]:
###encoding 'Billing_Country'

In [15]:
# Create mean target encoded feature
train['Billing_Country_enc'], test['Billing_Country_enc'] = mean_target_encoding(train=train,
                                                                     test=test,
                                                                     target='Stage',
                                                                     categorical='Billing_Country',
                                                                     alpha=10)

<ipython-input-8-5785e70fe0cb>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  train_feature = pd.Series(index=train.index)


In [16]:
# Create mean target encoded feature
train['Account_Type_enc'], test['Account_Type_enc'] = mean_target_encoding(train=train,
                                                                     test=test,
                                                                     target='Stage',
                                                                     categorical='Account_Type',
                                                                     alpha=10)

<ipython-input-8-5785e70fe0cb>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  train_feature = pd.Series(index=train.index)


In [17]:
Opportunity_Type

NameError: name 'Opportunity_Type' is not defined

In [18]:
train['Opportunity_Type_enc'], test['Opportunity_Type_enc'] = mean_target_encoding(train=train,
                                                                     test=test,
                                                                     target='Stage',
                                                                     categorical='Opportunity_Type',
                                                                     alpha=10)

<ipython-input-8-5785e70fe0cb>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  train_feature = pd.Series(index=train.index)


In [19]:
Account_Owner

NameError: name 'Account_Owner' is not defined

In [20]:
train['Account_Owner_enc'], test['Account_Owner_enc'] = mean_target_encoding(train=train,
                                                                     test=test,
                                                                     target='Stage',
                                                                     categorical='Account_Owner',
                                                                     alpha=10)

<ipython-input-8-5785e70fe0cb>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  train_feature = pd.Series(index=train.index)


In [21]:
Account_Name

NameError: name 'Account_Name' is not defined

In [22]:
train['Account_Name_enc'], test['Account_Name_enc'] = mean_target_encoding(train=train,
                                                                     test=test,
                                                                     target='Stage',
                                                                     categorical='Account_Name',
                                                                     alpha=10)

<ipython-input-8-5785e70fe0cb>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  train_feature = pd.Series(index=train.index)


In [23]:
train.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Total_Taxable_Amount,Stage,Prod_Category_A,Region_enc,Territory_enc,Billing_Country_enc,Account_Type_enc,Opportunity_Type_enc,Account_Owner_enc,Account_Name_enc
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,5272800.0,0,Prod_Category_A_None,0.528996,0.604688,0.566906,0.536191,0.527702,0.402094,0.643524
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,48230.0,1,Prod_Category_A_None,0.532181,0.609810,0.551745,0.540773,0.530896,0.387281,0.634526
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,83865.6,1,Prod_Category_A_None,0.453823,0.617116,0.461336,0.300891,0.527702,0.663810,0.800132
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,7421881.5,0,Prod_Category_A_None,0.455792,0.629018,0.462437,0.297194,0.212627,0.602113,0.399635
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,13357192.5,0,Prod_Category_A_None,0.453823,0.617116,0.461336,0.300891,0.212401,0.581201,0.330085


In [24]:
train.columns

Index(['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Source ',
       'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Opportunity_ID',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date', 'TRF',
       'Total_Amount_Currenc

In [25]:
train.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Total_Taxable_Amount,Stage,Prod_Category_A,Region_enc,Territory_enc,Billing_Country_enc,Account_Type_enc,Opportunity_Type_enc,Account_Owner_enc,Account_Name_enc
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,5272800.0,0,Prod_Category_A_None,0.528996,0.604688,0.566906,0.536191,0.527702,0.402094,0.643524
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,48230.0,1,Prod_Category_A_None,0.532181,0.609810,0.551745,0.540773,0.530896,0.387281,0.634526
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,83865.6,1,Prod_Category_A_None,0.453823,0.617116,0.461336,0.300891,0.527702,0.663810,0.800132
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,7421881.5,0,Prod_Category_A_None,0.455792,0.629018,0.462437,0.297194,0.212627,0.602113,0.399635
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,13357192.5,0,Prod_Category_A_None,0.453823,0.617116,0.461336,0.300891,0.212401,0.581201,0.330085


In [26]:
#Agrego las columnas con valores unicos a "Columnas_a_borrar"
Columnas_a_borrar = ['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Source ',
       'Billing_Country', 'Account_Name', 'Opportunity_Name',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date',
       'Total_Amount_Currency','Total_Taxable_Amount_Currency',
       'Prod_Category_A','Total_Amount' ]

In [27]:
#Borro las columnas train
train.drop(columns = Columnas_a_borrar, inplace=True)

In [28]:
#Borro las columnas test
test.drop(columns = Columnas_a_borrar, inplace=True)

In [29]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Opportunity_ID        16947 non-null  int64  
 1   TRF                   16947 non-null  int64  
 2   Total_Taxable_Amount  16947 non-null  float64
 3   Stage                 16947 non-null  int32  
 4   Region_enc            16947 non-null  float64
 5   Territory_enc         16947 non-null  float64
 6   Billing_Country_enc   16947 non-null  float64
 7   Account_Type_enc      16947 non-null  float64
 8   Opportunity_Type_enc  16947 non-null  float64
 9   Account_Owner_enc     16947 non-null  float64
 10  Account_Name_enc      16947 non-null  float64
dtypes: float64(8), int32(1), int64(2)
memory usage: 1.4 MB


In [30]:
for column in train.columns:
    if train[column].dtype == 'int32':
        train[column] = train[column].astype('float64')
print(train.dtypes)

Opportunity_ID            int64
TRF                       int64
Total_Taxable_Amount    float64
Stage                   float64
Region_enc              float64
Territory_enc           float64
Billing_Country_enc     float64
Account_Type_enc        float64
Opportunity_Type_enc    float64
Account_Owner_enc       float64
Account_Name_enc        float64
dtype: object


In [ ]:
####  Modelos

In [31]:
X = train.drop(['Stage'], axis=1)
y = train['Stage']

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
###Ramdon Forest

In [79]:
from sklearn.ensemble import RandomForestClassifier

In [80]:
rf_model = RandomForestClassifier(random_state=123)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=123)

In [81]:
preds = rf_model.predict_proba(X_test)

In [82]:
preds

array([[0.62, 0.38],
       [0.22, 0.78],
       [0.01, 0.99],
       ...,
       [0.05, 0.95],
       [0.88, 0.12],
       [0.73, 0.27]])

In [83]:
preds = preds[:, 1]

In [84]:
preds

array([0.38, 0.78, 0.99, ..., 0.95, 0.12, 0.27])

In [85]:
train.head(30)

,Opportunity_ID,TRF,Total_Taxable_Amount,Stage,Region_enc,Territory_enc,Billing_Country_enc,Account_Type_enc,Opportunity_Type_enc,Account_Owner_enc,Account_Name_enc
0,0,10,5272800.0,0.0,0.528996,0.604688,0.566906,0.536191,0.527702,0.402094,0.643524
1,1,0,48230.0,1.0,0.532181,0.609810,0.551745,0.540773,0.530896,0.387281,0.634526
2,2,0,83865.6,1.0,0.453823,0.617116,0.461336,0.300891,0.527702,0.663810,0.800132
3,3,14,7421881.5,0.0,0.455792,0.629018,0.462437,0.297194,0.212627,0.602113,0.399635
4,4,25,13357192.5,0.0,0.453823,0.617116,0.461336,0.300891,0.212401,0.581201,0.330085
5,5,28,14838277.5,0.0,0.460034,0.619184,0.467164,0.301841,0.217643,0.602172,0.332124
6,6,7,2659494.6,0.0,0.455792,0.629018,0.462437,0.297194,0.212627,0.602113,0.399635
7,7,0,50688.0,1.0,0.455792,0.629018,0.462437,0.297194,0.525710,0.660075,0.807932
8,8,0,470400.0,0.0,0.679257,0.604688,0.680904,0.536191,0.684690,0.651573,0.482220
9,8,0,470400.0,0.0,0.685132,0.610279,0.687219,0.542096,0.687868,0.655467,0.484174


In [86]:
rf_model.predict_proba(X)

array([[0.91, 0.09],
       [0.12, 0.88],
       [0.06, 0.94],
       ...,
       [0.02, 0.98],
       [0.93, 0.07],
       [0.97, 0.03]])

In [42]:
# Import log_loss from sklearn
from sklearn.metrics import log_loss

In [87]:
print('Sklearn LogLoss: {:.5f}'.format(log_loss(y_test, preds)))

Sklearn LogLoss: 0.30678


In [ ]:
### esta aca train ahora predigo en test

In [47]:
preds_test = rf_model.predict_proba(test)

In [88]:
preds_test = preds_test[:, 1]

In [89]:
preds_test

array([0.6 , 0.6 , 0.6 , ..., 0.35, 0.39, 0.21])

In [90]:
preds_test.shape

(2551,)

In [98]:
test['Target']=preds_test

In [100]:
test['Target']=preds_test
sub= test[['Opportunity_ID','Target']]
sub1=sub.groupby(['Opportunity_ID'], as_index=False).mean()
sub1[['Opportunity_ID', 'Target']].to_csv('submission2.csv', index=False)

In [101]:
sub

,Opportunity_ID,Target
0,10689,0.60
1,10689,0.60
2,10689,0.60
3,10690,0.25
4,10690,0.68
...,...,...
2546,12364,0.65
2547,12365,0.35
2548,12366,0.35
2549,12367,0.39


In [108]:
sub1=sub.groupby(['Opportunity_ID'], as_index=False).mean()

In [109]:
sub1.shape

(1567, 2)

In [110]:
sub1

,Opportunity_ID,Target
0,10689,0.600
1,10690,0.594
2,10691,0.800
3,10692,0.480
4,10693,0.960
...,...,...
1562,12364,0.650
1563,12365,0.350
1564,12366,0.350
1565,12367,0.390


In [111]:
sub1[['Opportunity_ID', 'Target']].to_csv('submission2.csv', index=False)

In [95]:
test

,Opportunity_ID,TRF,Total_Taxable_Amount,Region_enc,Territory_enc,Billing_Country_enc,Account_Type_enc,Opportunity_Type_enc,Account_Owner_enc,Account_Name_enc,target
0,10689,0,367419.0,0.529873,0.607935,0.566931,0.706639,0.701380,0.670123,0.501715,0.60
1,10689,0,367419.0,0.529873,0.607935,0.566931,0.706639,0.701380,0.670123,0.501715,0.60
2,10689,0,367419.0,0.529873,0.607935,0.566931,0.706639,0.701380,0.670123,0.501715,0.60
3,10690,1,757783.5,0.529873,0.607935,0.566931,0.706639,0.701380,0.670123,0.501715,0.25
4,10690,0,757783.5,0.529873,0.607935,0.566931,0.706639,0.701380,0.670123,0.501715,0.68
...,...,...,...,...,...,...,...,...,...,...,...
2546,12364,0,147750.0,0.529873,0.505443,0.592399,0.706639,0.530477,0.670123,0.619327,0.65
2547,12365,0,45054.9,0.454772,0.382953,0.461864,0.539541,0.530477,0.361980,0.539074,0.35
2548,12366,0,100122.0,0.454772,0.382953,0.461864,0.539541,0.530477,0.361980,0.539074,0.35
2549,12367,0,143220.0,0.454772,0.343669,0.461864,0.539541,0.683635,0.360420,0.562518,0.39


In [ ]:
###xgboost

In [113]:
xg_reg = xgb.XGBClassifier(objective ='reg:squaredlogerror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [117]:
xg_reg.fit(X_train,y_train)

C:\Users\Julia\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=10, n_jobs=4, num_parallel_tree=1,
              objective='reg:squaredlogerror', random_state=0, reg_alpha=10,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [119]:
preds = xg_reg.predict(X_test)

In [120]:
print('Sklearn LogLoss: {:.5f}'.format(log_loss(y_test, preds)))

Sklearn LogLoss: 8.09987


In [127]:
test= test.drop(['target'], axis=1)

In [128]:
test = test.drop(['Target'], axis=1)

KeyError: "['Target'] not found in axis"

In [129]:
preds_test = xg_reg.predict_proba(test)

In [130]:
test.head()

,Opportunity_ID,TRF,Total_Taxable_Amount,Region_enc,Territory_enc,Billing_Country_enc,Account_Type_enc,Opportunity_Type_enc,Account_Owner_enc,Account_Name_enc
0,10689,0,367419.0,0.529873,0.607935,0.566931,0.706639,0.70138,0.670123,0.501715
1,10689,0,367419.0,0.529873,0.607935,0.566931,0.706639,0.70138,0.670123,0.501715
2,10689,0,367419.0,0.529873,0.607935,0.566931,0.706639,0.70138,0.670123,0.501715
3,10690,1,757783.5,0.529873,0.607935,0.566931,0.706639,0.70138,0.670123,0.501715
4,10690,0,757783.5,0.529873,0.607935,0.566931,0.706639,0.70138,0.670123,0.501715


In [131]:
preds_test = preds_test[:, 1]

In [132]:
preds_test = preds_test[:, 1]
test['Target']=preds_test
sub= test[['Opportunity_ID','Target']]
sub1=sub.groupby(['Opportunity_ID'], as_index=False).mean()
sub1[['Opportunity_ID', 'Target']].to_csv('submission5.csv', index=False)

In [ ]:
###fin de submission

In [49]:
preds_test

array([[0.4 , 0.6 ],
       [0.4 , 0.6 ],
       [0.4 , 0.6 ],
       ...,
       [0.65, 0.35],
       [0.61, 0.39],
       [0.79, 0.21]])

In [72]:
preds_test 

array([[0.4 , 0.6 ],
       [0.4 , 0.6 ],
       [0.4 , 0.6 ],
       ...,
       [0.65, 0.35],
       [0.61, 0.39],
       [0.79, 0.21]])

In [74]:
# Define your own LogLoss function
def own_logloss(y_true, prob_pred):
  	# Find loss for each observation
    terms = y_true * np.log(prob_pred) + (1 - y_true) * np.log(1 - prob_pred)
    # Find mean over all observations
    err = np.mean(terms)   
    return -err


In [75]:
print('Your LogLoss: {:.5f}'.format(own_logloss(y_test, preds)))

<ipython-input-74-3858206bf2f4>:4: RuntimeWarning: divide by zero encountered in log
  terms = y_true * np.log(prob_pred) + (1 - y_true) * np.log(1 - prob_pred)
<ipython-input-74-3858206bf2f4>:4: RuntimeWarning: invalid value encountered in log
  terms = y_true * np.log(prob_pred) + (1 - y_true) * np.log(1 - prob_pred)


TypeError: unsupported format string passed to Series.__format__

In [39]:
preds.shape

(3390, 2)

In [40]:
y_test.shape

(3390,)

In [ ]:
preds = rf_model.predict(test)

In [ ]:
len(preds)

In [ ]:
test['Stage'] = preds

In [64]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [65]:
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

In [70]:
preds = rf_model.predict=(X_test)

In [67]:
preds

array([0.51, 0.84, 1.  , ..., 0.99, 0.08, 0.11])

In [68]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.296379


In [71]:
print('Sklearn LogLoss: {:.5f}'.format(log_loss(y_test, preds)))

ValueError: y_true and y_pred contain different number of classes 2, 10. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0. 1.]